<a href="https://colab.research.google.com/github/otnemrasordep/DadaGP-track-processing/blob/main/bandGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/bandGP


/content/drive/MyDrive/bandGP


In [4]:
!ls

filtered_dg-dg-b.zip  filtered_drums.zip  wandb


In [3]:
# Install necessary libraries
!pip install transformers datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Import required libraries
import wandb
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

In [5]:
# Initialize Weights & Biases
# wandb.login(relogin=True)
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
# Start a new W&B run with increased timeout
wandb.init(
    project="bandGP-finetuning-t5",
    entity="otnemrasordep-queen-mary-university-of-london",
    settings=wandb.Settings(init_timeout=120),  # Increase timeout to 120 seconds
)

wandb: Currently logged in as: otnemrasordep (otnemrasordep-queen-mary-university-of-london). Use `wandb login --relogin` to force relogin


In [11]:
# Extract INPUT
import zipfile
import os
from tqdm.notebook import tqdm

# Path to your ZIP file
zip_file_path = "/content/drive/MyDrive/bandGP/filtered_dg-dg-b.zip"
# Destination folder for extracted files
extract_to = "/content/drive/MyDrive/bandGP/"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    total_files = len(file_list)

    # Use tqdm to show progress
    with tqdm(total=total_files, desc="Extracting files") as pbar:
        for file in file_list:
            zip_ref.extract(file, extract_to)
            pbar.update(1)  # Update progress


Extracting files:   0%|          | 0/49574 [00:00<?, ?it/s]

In [15]:
len(os.listdir('/content/drive/MyDrive/bandGP/filtered_dg-dg-b'))

49572

In [17]:
!rm -r /content/drive/MyDrive/bandGP/filtered_drums

In [18]:
# Extract OUTPUT
import zipfile
import os
from tqdm.notebook import tqdm

# Path to your ZIP file
zip_file_path = "/content/drive/MyDrive/bandGP/filtered_drums.zip"
# Destination folder for extracted files
extract_to = "/content/drive/MyDrive/bandGP/"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    total_files = len(file_list)

    # Use tqdm to show progress
    with tqdm(total=total_files, desc="Extracting files") as pbar:
        for file in file_list:
            # print(file)
            zip_ref.extract(file, extract_to)
            pbar.update(1)  # Update progress

Extracting files:   0%|          | 0/49574 [00:00<?, ?it/s]

In [19]:
len(os.listdir('/content/drive/MyDrive/bandGP/filtered_drums'))

49572

In [21]:
import json

# Collect input-output pairs
# T5 expects a "Input: xxx; Output: yyy" format for every file
pairs = []

# Input and output directories
input_dir = "/content/drive/MyDrive/bandGP/filtered_dg-dg-b"
output_dir = "/content/drive/MyDrive/bandGP/filtered_drums"

# Ensure files are sorted for pairing
input_files = sorted(os.listdir(input_dir))
output_files = sorted(os.listdir(output_dir))

# Create pairs
for x_file, y_file in tqdm(zip(input_files, output_files)):
    x_path = os.path.join(input_dir, x_file)
    y_path = os.path.join(output_dir, y_file)

    with open(x_path, "r") as x, open(y_path, "r") as y:
        input_text = x.read().strip()
        output_text = y.read().strip()
        pairs.append(f"Input: {input_text}\nOutput: {output_text}")

with open("paired-dataset-dgdgb-drums.json", "w") as final:
	json.dump(pairs, final)

0it [00:00, ?it/s]

In [8]:
import json

with open("paired-dataset-dgdgb-drums.json", "r") as final:
	pairs = json.load(final)

In [30]:
from datasets import load_dataset

# Load the JSON dataset
dataset = load_dataset("json", data_files={"data": "paired-dataset-dgdgb-drums.json"})
dataset = dataset["data"]  # Access the dataset


Generating data split: 0 examples [00:00, ? examples/s]

In [31]:
# Split into training and validation (80-20 split)
split = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = split["train"]
validation_dataset = split["test"]


In [32]:
print(f'Training set: {len(train_dataset)} exampples')
print(f'Validation set: {len(validation_dataset)} exampples')

Training set: 39657 exampples
Validation set: 9915 exampples


In [29]:
print(dataset[0])

{'text': 'Input: artist:dgdgb\ndowntune:-3\ntempo:100\nstart\nnew_measure\ndistorted0:note:s6:f-2\ndistorted1:note:s6:f-2\nbass:note:s5:f-2\nwait:480\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s3:f10\ndistorted0:note:s4:f10\ndistorted0:note:s5:f8\ndistorted1:note:s3:f10\ndistorted1:note:s4:f10\ndistorted1:note:s5:f8\nbass:note:s4:f8\nwait:240\ndistorted0:note:s5:f0\nnfx:dead\ndistorted0:note:s6:f-2\nnfx:dead\ndistorted1:note:s5:f0\nnfx:dead\ndistorted1:note:s6:f-2\nnfx:dead\nbass:rest\nwait:240\ndistorted0:note:s5

## Tokenization for T5


In [13]:
from transformers import AutoTokenizer

# Load the T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Replace with your desired T5 model size


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [35]:
def preprocess_text_split(example):
    text = example["text"]
    if "Input:" in text and "Output:" in text:
        input_start = text.find("Input:") + len("Input:")
        output_start = text.find("Output:")
        input_text = text[input_start:output_start].strip()
        target_text = text[output_start + len("Output:"):].strip()
    else:
        input_text = ""
        target_text = ""
    return {"input_text": input_text, "target_text": target_text}

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_text_split)
validation_dataset = validation_dataset.map(preprocess_text_split)


Map:   0%|          | 0/9915 [00:00<?, ? examples/s]

In [36]:
validation_dataset[0]

{'text': 'Input: artist:dgdgb\ndowntune:-1\ntempo:130\nstart\nnew_measure\ndistorted0:rest\ndistorted1:rest\nbass:rest\nwait:1920\ndistorted0:rest\ndistorted1:rest\nwait:960\ndistorted0:rest\ndistorted1:rest\nwait:480\ndistorted0:note:s1:f7\ndistorted0:note:s2:f7\ndistorted0:note:s3:f8\ndistorted0:note:s4:f9\ndistorted0:note:s5:f9\ndistorted0:note:s6:f7\ndistorted1:note:s2:f12\ndistorted1:note:s3:f11\ndistorted1:note:s4:f9\nwait:240\ndistorted0:note:s1:f7\ndistorted0:note:s2:f7\ndistorted0:note:s3:f8\ndistorted0:note:s4:f9\ndistorted0:note:s5:f9\ndistorted0:note:s6:f7\ndistorted1:note:s2:f12\ndistorted1:note:s3:f11\ndistorted1:note:s4:f9\nwait:240\nnew_measure\ndistorted0:note:s1:f7\ndistorted0:note:s2:f7\ndistorted0:note:s3:f8\ndistorted0:note:s4:f9\ndistorted0:note:s5:f9\ndistorted0:note:s6:f7\ndistorted1:note:s2:f12\ndistorted1:note:s3:f11\ndistorted1:note:s4:f9\nbass:note:s4:f2\nwait:960\ndistorted0:note:s5:f11\ndistorted0:note:s6:f7\nbass:note:s4:f2\nwait:480\ndistorted0:note:s5:f

In [42]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenization function
def preprocess_function(examples):
    # Tokenize input_text
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=512,
        padding="max_length",
        truncation=True,
    )
    # Tokenize target_text as labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=512,
            padding="max_length",
            truncation=True,
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [43]:
# Apply tokenization
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True)


In [47]:
print(tokenized_train)

Dataset({
    features: ['text', 'input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 39657
})


In [44]:
tokenized_train.save_to_disk("/content/drive/MyDrive/bandGP/tokenized_train_t5_dgdgb-drums")
tokenized_validation.save_to_disk("/content/drive/MyDrive/bandGP/tokenized_validation_t5_dgdgb-drums")

Saving the dataset (0/4 shards):   0%|          | 0/39657 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9915 [00:00<?, ? examples/s]

In [ ]:
# To load tokenized datasets
from datasets import load_from_disk

tokenized_train = load_from_disk("/content/drive/MyDrive/bandGP/tokenized_train_t5_dgdgb-drums")
tokenized_validation = load_from_disk("/content/drive/MyDrive/bandGP/tokenized_validation_t5_dgdgb-drums")

In [45]:
import numpy as np

# Calculate token length statistics // TRAINING
token_lengths = [len(tokens["input_ids"]) for tokens in tokenized_train]
print(f"Max Length: {max(token_lengths)}")
print(f"Min Length: {min(token_lengths)}")
print(f"Median Length: {np.median(token_lengths)}")

# IDEAL FOR TRAINING!

Max Length: 512
Min Length: 512
Median Length: 512.0


In [27]:
print(tokenized_train['text'][0])

Input: artist:dgdgb
downtune:0
tempo:180
start
new_measure
distorted0:rest
distorted1:rest
bass:rest
wait:480
distorted0:note:s2:f8
distorted0:note:s3:f9
distorted0:note:s4:f9
distorted0:note:s5:f7
wait:480
distorted0:note:s2:f8
distorted0:note:s3:f9
distorted0:note:s4:f9
distorted0:note:s5:f7
wait:480
distorted0:note:s2:f0
nfx:dead
distorted0:note:s3:f0
nfx:dead
distorted0:note:s4:f0
nfx:dead
distorted0:note:s5:f0
nfx:dead
wait:480
distorted0:note:s2:f8
distorted0:note:s3:f9
distorted0:note:s4:f9
distorted0:note:s5:f7
wait:480
distorted0:note:s2:f8
distorted0:note:s3:f9
distorted0:note:s4:f9
distorted0:note:s5:f7
wait:480
distorted0:note:s2:f8
distorted0:note:s3:f9
distorted0:note:s4:f9
distorted0:note:s5:f7
wait:480
distorted0:note:s2:f0
nfx:dead
distorted0:note:s3:f0
nfx:dead
distorted0:note:s4:f0
nfx:dead
distorted0:note:s5:f0
nfx:dead
wait:480
new_measure
distorted0:note:s2:f8
distorted0:note:s3:f9
distorted0:note:s4:f9
distorted0:note:s5:f7
distorted1:rest
bass:rest
wait:480
dist

In [25]:
# Print the entire structure of the first example
print(tokenized_train[0])

# Alternatively, loop through keys and values for better clarity
for key, value in tokenized_train[0].items():
    print(f"{key}")  # Print only the first 10 items for brevity (if applicable)


{'text': 'Input: artist:dgdgb\ndowntune:0\ntempo:180\nstart\nnew_measure\ndistorted0:rest\ndistorted1:rest\nbass:rest\nwait:480\ndistorted0:note:s2:f8\ndistorted0:note:s3:f9\ndistorted0:note:s4:f9\ndistorted0:note:s5:f7\nwait:480\ndistorted0:note:s2:f8\ndistorted0:note:s3:f9\ndistorted0:note:s4:f9\ndistorted0:note:s5:f7\nwait:480\ndistorted0:note:s2:f0\nnfx:dead\ndistorted0:note:s3:f0\nnfx:dead\ndistorted0:note:s4:f0\nnfx:dead\ndistorted0:note:s5:f0\nnfx:dead\nwait:480\ndistorted0:note:s2:f8\ndistorted0:note:s3:f9\ndistorted0:note:s4:f9\ndistorted0:note:s5:f7\nwait:480\ndistorted0:note:s2:f8\ndistorted0:note:s3:f9\ndistorted0:note:s4:f9\ndistorted0:note:s5:f7\nwait:480\ndistorted0:note:s2:f8\ndistorted0:note:s3:f9\ndistorted0:note:s4:f9\ndistorted0:note:s5:f7\nwait:480\ndistorted0:note:s2:f0\nnfx:dead\ndistorted0:note:s3:f0\nnfx:dead\ndistorted0:note:s4:f0\nnfx:dead\ndistorted0:note:s5:f0\nnfx:dead\nwait:480\nnew_measure\ndistorted0:note:s2:f8\ndistorted0:note:s3:f9\ndistorted0:note:s4